# Criando um dataset com as letras das 200 músicas mais tocadas no Spotify

O objetivo desse notebook foi coletar as letras das 200 músicas mais tocadas no Spotify na primeira semana de novembro, para o Brasil, de acordo com a lista gerada pelo [Spotify Charts](https://spotifycharts.com/regional/br/weekly/2019-11-01--2019-11-08)

# 1. Coletando dados do Genius API usando a biblioteca LyricsGenius


## Sobre a biblioteca LyricsGenius:

De acordo com o documento disponível no [GitHub](https://github.com/johnwmillr/LyricsGenius): 
>"LyricsGenius provides a simple interface to the song, artist, and lyrics data stored on Genius.com."

In [1]:
# realizando a conexão com a biblioteca

import lyricsgenius

genius = lyricsgenius.Genius(
    "FCPDxhJ92xLxKh5hfCG-HVgweTxgk_OHbx3T-ok3vVn_9jQROwG3sQwMf9T1JO8g") # Client Access Token

In [2]:
# abrindo o arquivo csv baixado no site do Spotify com as músicas que serão coletadas

top200 = pd.read_csv('top200-nov.csv')
top200.drop(['Position','Streams', 'ID'], axis=1, inplace=True)
top200.head()

,Track,Artist
0,Surtada,Dadá Boladão
1,Gaiola É o Troco,MC Du Black
2,Some que ele vem atrás,Anitta
3,Supera,Marília Mendonça
4,Hit Contagiante,Felipe Original


Apliquei um for loop que faz a consulta de acordo com o nome da música e do artista e anexa os resultados à lista 'lyr'. Além disso, implementei uma verificação para quando a letra da música não fosse encontrada (para não causar problemas).

In [3]:
lyr = []

for i in range(len(top200)):
    track = top200.Track[i]
    artist = top200.Artist[i]
    
    try:
        song = genius.search_song(track, artist) # procura por uma música única de um artista específico
        genius.remove_section_headers = True # remove cabeçalhos da seção(ex. [Refrão]) das letras 
        lyr.append(song.lyrics)
        
    except:
        lyr.append('Não disponível')

Searching for "Surtada" by Dadá Boladão...
Done.
Searching for "Gaiola É o Troco" by MC Du Black...
Done.
Searching for "Some que ele vem atrás" by Anitta...
Done.
Searching for "Supera" by Marília Mendonça...
Done.
Searching for "Hit Contagiante" by Felipe Original...
Done.
Searching for "Chama Ela" by Lexa...
Done.
Searching for "quando a bad bater" by Luan Santana...
Done.
Searching for "Cheirosa" by Jorge & Mateus...
Done.
Searching for "Contatinho" by Leo Santana...
Done.
Searching for "Tijolão" by Jorge & Mateus...
Done.
Searching for "Ferida Curada" by Zé Neto & Cristiano...
Done.
Searching for "Invocada" by Ludmilla...
Done.
Searching for "Lençol Dobrado" by João Gustavo e Murilo...
Done.
Searching for "Sentou e Gostou" by MC JottaPê...
Done.
Searching for "Quarta Cadeira" by Matheus & Kauan...
Done.
Searching for "Quando a vontade bater" by Pk...
Done.
Searching for "Kenny G" by Matuê...
Done.
Searching for "Complicado" by Vitão...
Done.
Searching for "Serenata" by Marília Men

Done.
Searching for "Cem Mil" by Gusttavo Lima...
Done.
Searching for "Onze e Pouquinho" by Dilsinho...
No results found for: 'Onze e Pouquinho Dilsinho'
Searching for "Edredom" by Vitão...
Done.
Searching for "Memories" by Maroon 5...
Done.
Searching for "Ponte Perra" by Pabllo Vittar...
Done.
Searching for "Hoje Eu Vou Parar Na Gaiola" by Mc Livinho...
Done.
Searching for "Lights Up" by Harry Styles...
Done.
Searching for "S de Saudade" by Luíza & Maurílio...
Done.
Searching for "Espelho Meu" by Henrique & Diego...
Done.
Searching for "Indomável" by Pk...
Done.
Searching for "Oi" by Lagum...
Done.
Searching for "7 rings" by Ariana Grande...
Done.
Searching for "Piece Of Your Heart" by MEDUZA...
Done.
Searching for "Bem Pior Que Eu" by Marília Mendonça...
Done.
Searching for "Melhor Saída" by Lauana Prado...
Done.
Searching for "Balão" by Orochi...
Done.
Searching for "Shallow" by Lady Gaga...
Done.
Searching for "Outro Dia" by A Banca Records...
Done.
Searching for "It's You" by Ali 

In [4]:
# criando um dataframe com as letras extraídas

lyrics = pd.DataFrame(lyr)
lyrics.columns = ['Lyrics']

In [5]:
# juntando as letras aos nomes da música e do artista

df_lyrics = top200.join(lyrics)

In [6]:
df_lyrics.head(10)

,Track,Artist,Lyrics
0,Surtada,Dadá Boladão,"[Letra de ""Surtada (Remix)"" com Dadá Boladão, ..."
1,Gaiola É o Troco,MC Du Black,"Gaiola é o troco\nDu Black na voz, na voz, na ..."
2,Some que ele vem atrás,Anitta,Amiga do céu\nMeu boy me largou\nParou de segu...
3,Supera,Marília Mendonça,Tá de novo com essa pessoa\nNão tô acreditand...
4,Hit Contagiante,Felipe Original,"Ahn, fala tu que eu tô cansado\nFelipe Origina..."
5,Chama Ela,Lexa,"1\n• ""C Walk"", YFG Borges (part. IAMBINHU & YF..."
6,quando a bad bater,Luan Santana,"Amor, eu só quero seu bem\nSe for embora agora..."
7,Cheirosa,Jorge & Mateus,"1\n• ""Meu Drip"", Lucas Muto (part. Viper)\n• ""..."
8,Contatinho,Leo Santana,"1\n• ""C Walk"", YFG Borges (part. IAMBINHU & YF..."
9,Tijolão,Jorge & Mateus,Eu te conheci depois que a gente largou\nSerá ...


Olhando o topo do dataframe pode-se observar que algumas letras não foram extraídas corretamente

In [7]:
# analisando a música Chama Ela, da Lexa, que parece ter sido extraída de forma errada

print(df_lyrics.iloc[5]['Lyrics'])

1
• "C Walk", YFG Borges (part. IAMBINHU & YFG Rush)
2
• "Codinome", Maria Andrade
• "Mano Não Toca Na Lace (Remix)", Azzy (part. Ludmilla & DJ Will 22)
• "Sentença", Duzz (part. Klyn)
3
• "Curtir a Vida", DJ Dollynho da Lapa (part. MC Cabelinho)
• "Fah Fah Fah Fah", BC Raff (part. Loc Dog)
• "Lupa", Sos
• "Real", DaPaz (part. Azzy, Chris MC & Felp22)
4
• "99", Massaru
• "Kenny G", Matuê
• "Senta Como Se Não Houvesse Amanhã", Kevin o Chris (part. MC Calvin & MC Alysson)
• "Wish", IAMBINHU (part. Ebony & VXNXMS)
5
• "Criminal", ADL (part. Filipe Ret)
• "Jovem Tralha", Felp22 & Meno Tody
6
• "Contatinho", Léo Santana (part. Anitta)
• "Malvadão 2", Xamã
• "Mirando Alto", Burn-O
• "Nunca Vi", MC Leen (part. ML & Derek)
• "Overdrive (Remix)", Willsbife
• "Prada", ZAPI (part. Derek)
• "Sauce", Lil Preto
• "Solta a Ousadia", Tainá Costa
7
• "Os Guri da PlanoAlto", Plano Alto (part. França, @pfcarter & AkaLet)
• "Penny", Massaru
8
• "PLAYBOY BUNNY!!", virgingod (part. Lil Bice)
9
• "Flash", MC

In [8]:
# filtrando as músicas com letras extraídas erradas ou não encontradas

mask1 = df_lyrics[df_lyrics['Lyrics'].str.contains('1\n')]
mask2 = df_lyrics[df_lyrics['Lyrics']=='Não disponível']

In [9]:
print(mask1.index, mask2.index)

Int64Index([5, 7, 8, 25, 33, 44, 68, 75, 79, 100, 102, 123, 130, 144, 181], dtype='int64') Int64Index([ 27,  29,  35,  38,  47,  51,  61,  62,  69,  70,  85,  97, 112,
            118, 132, 135, 138, 165, 169, 178, 187],
           dtype='int64')


In [10]:
# salvando as músicas que ficaram sem a letra correta em um dataframe

df_no_lyrics = pd.DataFrame(df_lyrics.iloc[[5, 7, 8, 25, 33, 44, 68, 75, 79, 100, 102, 123, 130, 144, 181, 27,  29,  
                                            35, 38, 47,  51,  61,  62,  69,  70,  85,  97, 112, 118, 132, 135, 138, 
                                            165, 169, 178, 187]])
df_no_lyrics.head()

,Track,Artist,Lyrics
5,Chama Ela,Lexa,"1\n• ""C Walk"", YFG Borges (part. IAMBINHU & YF..."
7,Cheirosa,Jorge & Mateus,"1\n• ""Meu Drip"", Lucas Muto (part. Viper)\n• ""..."
8,Contatinho,Leo Santana,"1\n• ""C Walk"", YFG Borges (part. IAMBINHU & YF..."
25,Eterna Sacanagem,MC JottaPê,"1\n• ""Meu Drip"", Lucas Muto (part. Viper)\n• ""..."
33,Resenha lá em casa,MC Kevin o Chris,"1\n• ""BOLT"", Febem\n• ""Foda-se a Ex"", MC Cabel..."


In [11]:
# excluindo coluna Lyrics, pois seus valores não são úteis, já que não foram extraídos corretamente

df_no_lyrics.drop(['Lyrics'], axis=1, inplace=True)

In [12]:
# excluindo do dataframe as letras que foram extraídas erradas

df_lyrics.drop(index = [5, 7, 8, 25, 33, 44, 68, 75, 79, 100, 102, 123, 130, 144, 181, 27,  29,  35,  38,  47,  51,  
                        61,  62,  69,  70,  85,  97, 112, 118, 132, 135, 138, 165, 169, 178, 187], inplace=True)
df_lyrics.head()

,Track,Artist,Lyrics
0,Surtada,Dadá Boladão,"[Letra de ""Surtada (Remix)"" com Dadá Boladão, ..."
1,Gaiola É o Troco,MC Du Black,"Gaiola é o troco\nDu Black na voz, na voz, na ..."
2,Some que ele vem atrás,Anitta,Amiga do céu\nMeu boy me largou\nParou de segu...
3,Supera,Marília Mendonça,Tá de novo com essa pessoa\nNão tô acreditand...
4,Hit Contagiante,Felipe Original,"Ahn, fala tu que eu tô cansado\nFelipe Origina..."


In [13]:
print('A primeira etapa teve como resultado %d letras coletadas corretamente e %d letras que ainda precisam ser coletadas' % (len(df_lyrics), len(df_no_lyrics)))

A primeira etapa teve como resultado 164 letras coletadas corretamente e 36 letras que ainda precisam ser coletadas


# 2. Web Scraping para coletar as letras que faltaram

Usando a biblioteca Beautiful Soup no site [Letras.mus.br](https://www.letras.mus.br/).

In [14]:
# importando bibliotecas

import requests
from bs4 import BeautifulSoup

In [15]:
# função para automatizar o web scraping

def scraper_music(artist_r, track_r):
    
    try:
        url = 'https://www.letras.mus.br/' + str(artist_r).lower().replace(' ', '-').replace('&', '') + '/' + str(track_r).lower().replace(' ', '-') + '/'
        data = requests.get(url)
        soup = BeautifulSoup(data.content, 'html.parser')
    
        lyr_r = soup.find_all('div', {'class' : 'cnt-letra p402_premium'})
        
        lyrics = []

        for item in lyr_r[0].find_all('p'):
            item = str(item)
            item = item.replace('<br>', '\n').replace('<br/>', '\n').replace('</br>', '').replace('</p>', '').replace('<p>', '')
            lyrics.append(item)
            
        df_lyrics_r = pd.DataFrame({'Track': track_r, 'Artist': artist_r, 'Lyrics': '\n'.join(lyrics)}, index=[0])
        return df_lyrics_r
        
    except:
        return pd.DataFrame({'Track': track_r, 'Artist': artist_r, 'Lyrics': 'Não disponível'}, index=[0])

In [16]:
# interando todos os artistas e músicas pela função de web scraping com um for loop

df_no_lyrics = df_no_lyrics.reset_index(drop=True)
df_scrap = pd.DataFrame(columns = ['Track', 'Artist', 'Lyrics'])

for i in range(len(df_no_lyrics)):
    track_rem = df_no_lyrics.Track[i]
    artist_rem = df_no_lyrics.Artist[i]
    df_scrap = df_scrap.append(scraper_music(artist_rem, track_rem))

In [17]:
# filtrando quais músicas continuam sem a letra

df_scrap = df_scrap.reset_index(drop=True)
df_naodisp = df_scrap[df_scrap['Lyrics']=='Não disponível']
df_naodisp.index

Int64Index([13, 17, 19, 20, 22, 25, 33, 34], dtype='int64')

In [18]:
# exlcuíndo do df_scrap as músicas que continuam sem a letra

df_scrap.drop(index = [13, 17, 19, 20, 22, 25, 33, 34], inplace=True)

# 3. Extração manual das letras que estão com padrões de URL diferentes

Ainda usando a biblioteca Beautiful Soup no site [Letras.mus.br](https://www.letras.mus.br/).

In [19]:
df_naodisp

,Track,Artist,Lyrics
13,S de Saudade,Luíza & Maurílio,Não disponível
17,3 Batidas,Guilherme & Benuto,Não disponível
19,Também Sei Fazer Falta,Israel & Rodolffo,Não disponível
20,Relógio Parado,Diego & Arnaldo,Não disponível
22,Nem de Graça / Saudade Arregaça,Pixote,Não disponível
25,Flor Que Se Cheira,Guilherme & Benuto,Não disponível
33,Regras,Diego & Arnaldo,Não disponível
34,Show De Recaída,Bruno & Marrone,Não disponível


- S de Saudade, Luíza & Maurílio

In [21]:
data1 = requests.get('https://www.letras.mus.br/luiza-e-maurilio/s-de-saudade-part-ze-neto-e-cristiano/')
soup1 = BeautifulSoup(data1.content, 'html.parser')
    
lyr1 = soup1.find_all('div', {'class' : 'cnt-letra p402_premium'})

lyrics1 = []

for item in lyr1[0].find_all('p'):
    item = str(item)
    item = item.replace('<br>', '\n').replace('<br/>', '\n').replace('</br>', '').replace('</p>', '').replace('<p>', '')
    lyrics1.append(item)
    
df_lyrics_1 = pd.DataFrame({'Track': 'S de Saudade', 'Artist': 'Luíza & Maurílio', 'Lyrics': '\n'.join(lyrics1)}, index=[0])
    
df_lyrics_1

,Track,Artist,Lyrics
0,S de Saudade,Luíza & Maurílio,"É, olha eu aqui, na vida de solteiro que eu se..."


- 3 Batidas, Guilherme & Benuto

In [22]:
data2 = requests.get('https://www.letras.mus.br/guilherme-e-benuto/3-batidas-amando-bebendo-e-sofrendo/')
soup2 = BeautifulSoup(data2.content, 'html.parser')
    
lyr2 = soup2.find_all('div', {'class' : 'cnt-letra p402_premium'})

lyrics2 = []

for item in lyr2[0].find_all('p'):
    item = str(item)
    item = item.replace('<br>', '\n').replace('<br/>', '\n').replace('</br>', '').replace('</p>', '').replace('<p>', '')
    lyrics2.append(item)
    
df_lyrics_2 = pd.DataFrame({'Track': '3 Batidas', 'Artist': 'Guilherme & Benuto', 'Lyrics': '\n'.join(lyrics2)}, index=[0])
    
df_lyrics_2

,Track,Artist,Lyrics
0,3 Batidas,Guilherme & Benuto,"Eu tô rindo, mas é desespero\nTô mais perdido ..."


- Também Sei Fazer Falta, Israel & Rodolffo

In [23]:
data3 = requests.get('https://www.letras.mus.br/israel-rodolfo/tambem-sei-fazer-falta/')
soup3 = BeautifulSoup(data3.content, 'html.parser')
    
lyr3 = soup3.find_all('div', {'class' : 'cnt-letra p402_premium'})

lyrics3 = []

for item in lyr3[0].find_all('p'):
    item = str(item)
    item = item.replace('<br>', '\n').replace('<br/>', '\n').replace('</br>', '').replace('</p>', '').replace('<p>', '')
    lyrics3.append(item)
    
df_lyrics_3 = pd.DataFrame({'Track': 'Também Sei Fazer Falta', 'Artist': 'Israel & Rodolffo', 'Lyrics': '\n'.join(lyrics3)}, index=[0])
    
df_lyrics_3

,Track,Artist,Lyrics
0,Também Sei Fazer Falta,Israel & Rodolffo,Uma pessoa falou que sua pessoa tá pensando em...


- Relógio Parado, Diego & Arnaldo

In [24]:
data4 = requests.get('https://www.letras.mus.br/diego-e-arnaldo/relogio-parado/')
soup4 = BeautifulSoup(data4.content, 'html.parser')
    
lyr4 = soup4.find_all('div', {'class' : 'cnt-letra p402_premium'})

lyrics4 = []

for item in lyr4[0].find_all('p'):
    item = str(item)
    item = item.replace('<br>', '\n').replace('<br/>', '\n').replace('</br>', '').replace('</p>', '').replace('<p>', '')
    lyrics4.append(item)
    
df_lyrics_4 = pd.DataFrame({'Track': 'Relógio Parado', 'Artist': 'Diego & Arnaldo', 'Lyrics': '\n'.join(lyrics4)}, index=[0])
    
df_lyrics_4

,Track,Artist,Lyrics
0,Relógio Parado,Diego & Arnaldo,Ela não sabe ainda\nMas eu vou ser o grande am...


- Nem de Graça / Saudade Arregaça, Pixote

In [28]:
data5 = requests.get('https://www.letras.mus.br/pixote/nem-de-graca/')
soup5 = BeautifulSoup(data5.content, 'html.parser')
    
lyr5 = soup5.find_all('div', {'class' : 'cnt-letra p402_premium'})

lyrics5 = []

for item in lyr5[0].find_all('p'):
    item = str(item)
    item = item.replace('<br>', '\n').replace('<br/>', '\n').replace('</br>', '').replace('</p>', '').replace('<p>', '')
    lyrics5.append(item)
    
df_lyrics_5 = pd.DataFrame({'Track': 'Nem de Graça / Saudade Arregaça', 'Artist': 'Pixote', 'Lyrics': '\n'.join(lyrics5)}, index=[0])
    
df_lyrics_5

,Track,Artist,Lyrics
0,Nem de Graça / Saudade Arregaça,Pixote,É que não sobrou espaço pra outro alguém\nMinh...


- Flor Que Se Cheira, Guilherme & Benuto

In [29]:
data6 = requests.get('https://www.letras.mus.br/guilherme-e-benuto/flor-que-se-cheira/')
soup6 = BeautifulSoup(data6.content, 'html.parser')
    
lyr6 = soup6.find_all('div', {'class' : 'cnt-letra p402_premium'})

lyrics6 = []

for item in lyr6[0].find_all('p'):
    item = str(item)
    item = item.replace('<br>', '\n').replace('<br/>', '\n').replace('</br>', '').replace('</p>', '').replace('<p>', '')
    lyrics6.append(item)
    
df_lyrics_6 = pd.DataFrame({'Track': 'Flor Que Se Cheira', 'Artist': 'Guilherme & Benuto', 'Lyrics': '\n'.join(lyrics6)}, index=[0])
    
df_lyrics_6

,Track,Artist,Lyrics
0,Flor Que Se Cheira,Guilherme & Benuto,Cê sabe que o povo comenta sempre\nQue eu pres...


- Regras, Diego & Arnaldo

In [25]:
data7 = requests.get('https://www.letras.mus.br/diego-e-arnaldo/regras/')
soup7 = BeautifulSoup(data7.content, 'html.parser')
    
lyr7 = soup7.find_all('div', {'class' : 'cnt-letra p402_premium'})

lyrics7 = []

for item in lyr7[0].find_all('p'):
    item = str(item)
    item = item.replace('<br>', '\n').replace('<br/>', '\n').replace('</br>', '').replace('</p>', '').replace('<p>', '')
    lyrics7.append(item)
    
df_lyrics_7 = pd.DataFrame({'Track': 'Regras', 'Artist': 'Diego & Arnaldo', 'Lyrics': '\n'.join(lyrics7)}, index=[0])
    
df_lyrics_7

,Track,Artist,Lyrics
0,Regras,Diego & Arnaldo,Tava aqui combinando com o meu coração\nEle tá...


- Show De Recaída, Bruno & Marrone

In [26]:
data8 = requests.get('https://www.letras.mus.br/bruno-e-marrone/show-de-recaida/')
soup8 = BeautifulSoup(data8.content, 'html.parser')
    
lyr8 = soup8.find_all('div', {'class' : 'cnt-letra p402_premium'})

lyrics8 = []

for item in lyr8[0].find_all('p'):
    item = str(item)
    item = item.replace('<br>', '\n').replace('<br/>', '\n').replace('</br>', '').replace('</p>', '').replace('<p>', '')
    lyrics8.append(item)
    
df_lyrics_8 = pd.DataFrame({'Track': 'Show De Recaída', 'Artist': 'Bruno & Marrone', 'Lyrics': '\n'.join(lyrics8)}, index=[0])
    
df_lyrics_8

,Track,Artist,Lyrics
0,Show De Recaída,Bruno & Marrone,Se arrependimento matasse\nEu já tinha morrido...


In [30]:
df_scrap_manual = pd.concat([df_lyrics_1, df_lyrics_2, df_lyrics_3, df_lyrics_4, df_lyrics_5, 
                             df_lyrics_6, df_lyrics_7, df_lyrics_8])
df_scrap_manual

,Track,Artist,Lyrics
0,S de Saudade,Luíza & Maurílio,"É, olha eu aqui, na vida de solteiro que eu se..."
0,3 Batidas,Guilherme & Benuto,"Eu tô rindo, mas é desespero\nTô mais perdido ..."
0,Também Sei Fazer Falta,Israel & Rodolffo,Uma pessoa falou que sua pessoa tá pensando em...
0,Relógio Parado,Diego & Arnaldo,Ela não sabe ainda\nMas eu vou ser o grande am...
0,Nem de Graça / Saudade Arregaça,Pixote,É que não sobrou espaço pra outro alguém\nMinh...
0,Flor Que Se Cheira,Guilherme & Benuto,Cê sabe que o povo comenta sempre\nQue eu pres...
0,Regras,Diego & Arnaldo,Tava aqui combinando com o meu coração\nEle tá...
0,Show De Recaída,Bruno & Marrone,Se arrependimento matasse\nEu já tinha morrido...


# 4. Juntando e salvando os dataframes das três etapas anteriores

In [31]:
df_letras = pd.concat([df_lyrics, df_scrap, df_scrap_manual])
df_letras = df_letras.reset_index(drop=True)
df_letras.head()

,Track,Artist,Lyrics
0,Surtada,Dadá Boladão,"[Letra de ""Surtada (Remix)"" com Dadá Boladão, ..."
1,Gaiola É o Troco,MC Du Black,"Gaiola é o troco\nDu Black na voz, na voz, na ..."
2,Some que ele vem atrás,Anitta,Amiga do céu\nMeu boy me largou\nParou de segu...
3,Supera,Marília Mendonça,Tá de novo com essa pessoa\nNão tô acreditand...
4,Hit Contagiante,Felipe Original,"Ahn, fala tu que eu tô cansado\nFelipe Origina..."


In [32]:
# conferindo se está com as 200 músicas

len(df_letras)

200

In [33]:
# salvando em um arquivo csv

df_letras.to_csv('letras_top200.csv', sep = '|', encoding='utf-8', index_label = False)